In [1]:
# Order preparation for NSE

# STATUS: Completed
# Run-time: 10 seconds

# Dependencies:
# /zdata/*.pkl - for pickles generated by 01_nse_scan program

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 7496, clientId=11) # kavi tws live
# ib = IB().connect('127.0.0.1', 4001, clientId=11) # kavi IBG live
# ib = IB().connect('127.0.0.1', 7499, clientId=11) # kavi tws paper-trade
# ib = IB().connect('127.0.0.1', 4004, clientId=11) # kavi IBG paper-trade

In [2]:
# %%time
import pandas as pd
import numpy as np
import datetime
from os import listdir

#... set limits and penalties
    
m_maxp = 0.015    # % of max margin allowed on net liquidity per scrip to limit positon risk
    
min_rom = 0.5     # 0.5 would be 50% return

base = 0.05       # Upper or Lower base multiple for prices

max_nlvp = 0.8    # max allowable nlv to prevent overall portfolio risk. 0.8 means 80% of NLV.
                  # max available funds for option trades = max_nlvp * NLV - initMargin

#... read the account info
ac = ib.accountSummary()
df_a = util.df(ac)

#... set max margin per position
net_liq = float(df_a[df_a.tag == 'NetLiquidation'].iloc[0].value) 
av_funds = float(df_a[df_a.tag == 'FullAvailableFunds'].iloc[0].value)
max_p = net_liq*m_maxp

#****    PREPARE TO HARVEST   ****
#_________________________________

#... read the positions
ps = ib.portfolio()
df_p = util.df(ps)

df_p['ibSymbol'] = [s.symbol for s in df_p.contract.values]

# get expected price percentage from DTE
def expPricePct(expiry):
    '''Gets expected price percentage from DTE for harvesting trades.
    Assumes max DTE to be 30 days.
    Arg: (expiry) as string 'yyymmdd', e.g. from expPricePct 
    Returns: expected price percentage (xpp) as float
    Ref: http://interactiveds.com.au/software/Linest-poly.xls ... for getting curve function
    '''
#     if dte is to be extracted from contract.lastTradeDateOrContractMonth
    dte = (util.parseIBDatetime(expiry) - datetime.datetime.now().date()).days
    
    if dte > 30:
        dte = 30  # Forces the max DTE to be 30 days
    
    xpp = (103.6008 - 3.63457*dte + 0.03454677*dte*dte)/100
    
    return xpp

# get the harvest as lower of discount from curve * averageCost and discount * marketPrice

expiry = [d.lastTradeDateOrContractMonth for d in df_p.contract]
discount = [m for m in map(expPricePct, expiry)]
df_p['hvstPrice'] = pd.concat([df_p.averageCost*discount, 
                               df_p.marketPrice*(1-np.array(discount))], axis=1).min(axis=1)

df_p.hvstPrice = np.floor(df_p.hvstPrice/base)*base # round down to the nearest 0.05

df_p.loc[df_p.hvstPrice == 0, 'hvstPrice'] = base  # make the 0s to 5 paise

# harvest open positions with hvstPrice
qual_contracts = ib.qualifyContracts(*df_p.contract)
df_p['qual_contracts'] = qual_contracts
df_p['harvestOrder'] = [LimitOrder(action='BUY', totalQuantity=-position, lmtPrice=hvstPrice) for position, hvstPrice in zip(df_p.position, df_p.hvstPrice)]

In [4]:
#****   PREPARE TO SOW !    ****
#________________________________

#... get the lots and margins
# from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

# Rename Symbol and Margin fields
df_paisa = df_paisa.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct'})

# Convert columns to numeric and make margin to pct
df_paisa = df_paisa.apply(pd.to_numeric, errors='ignore')
df_paisa.marginpct = df_paisa.marginpct.div(100)

# Truncate to 9 characters for ibSymbol
df_paisa['ibSymbol'] = df_paisa.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MM', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_paisa.ibSymbol = df_paisa.ibSymbol.replace(ntoi)

df_slm = pd.merge(df_p, df_paisa[['ibSymbol', 'Mlot', 'TotMgnPerShr']])

#... make the blacklist

df1 = df_slm.groupby('ibSymbol').sum()
df1['used_margin'] = -df1.position * df1.TotMgnPerShr
df1['max_margin'] = max_p
df1['avail_margin'] = df1.position * df1.TotMgnPerShr + df1.max_margin

df1['max_units'] = (df1.avail_margin/(df1.Mlot*df1.TotMgnPerShr)).apply(np.floor)

blacklist = list(df1[df1.max_units <= 0].index)

In [6]:
#... build the high-pop-roc dataframe

# read the dataframe pickles from zdata. Ignore the underscores (underlying)
fs = listdir('./zdata/')
fs = [f for f in fs if (f[-3:] == 'pkl') & (f[0] != '_')]
dfs = pd.concat([pd.read_pickle(r'./zdata/'+f) for f in fs], axis=0).reset_index(drop=True).sort_values('rom', ascending=False)

In [10]:
dfs

,symbol,strike,dte,right,undPrice,pop,rom,expPrice,hv,bsmDelta,bsmPrice,ibiv,ibdelta,ibprice,close,bid,ask,lot,margin,opt_ticker
448,BANKNIFTY,24900.0,1,P,27392.50,NaN,0.867495,9.50,0.000000,NaN,NaN,0.507343,-0.000505,0.097707,1.15,0.05,1.65,20,55193.3900,"Ticker(contract=Option(conId=340252363, symbol..."
449,BANKNIFTY,24800.0,1,P,27392.50,NaN,0.858364,9.40,0.000000,NaN,NaN,0.526060,-0.000456,0.090877,1.05,0.05,1.65,20,55193.3900,"Ticker(contract=Option(conId=340252353, symbol..."
458,BANKNIFTY,23900.0,1,P,27392.50,NaN,0.844666,9.25,0.000000,NaN,NaN,0.699465,-0.000231,0.058821,0.90,0.15,0.25,20,55193.3900,"Ticker(contract=Option(conId=340252282, symbol..."
445,BANKNIFTY,25200.0,1,P,27392.50,NaN,0.840101,9.20,0.000000,NaN,NaN,0.449915,-0.000685,0.119964,0.85,0.25,0.95,20,55193.3900,"Ticker(contract=Option(conId=340252387, symbol..."
447,BANKNIFTY,25000.0,1,P,27392.50,NaN,0.835535,9.15,0.000000,NaN,NaN,0.488653,-0.000562,0.105515,0.80,0.70,0.75,20,55193.3900,"Ticker(contract=Option(conId=340252368, symbol..."
457,BANKNIFTY,24000.0,1,P,27392.50,NaN,0.830969,9.10,0.000000,NaN,NaN,0.678634,-0.000240,0.059364,0.75,0.30,0.60,20,55193.3900,"Ticker(contract=Option(conId=340252288, symbol..."
446,BANKNIFTY,25100.0,1,P,27392.50,NaN,0.821838,9.00,0.000000,NaN,NaN,0.469402,-0.000620,0.112663,0.70,0.40,0.65,20,55193.3900,"Ticker(contract=Option(conId=340252378, symbol..."
451,BANKNIFTY,24600.0,1,P,27392.50,NaN,0.817272,8.95,0.000000,NaN,NaN,0.562641,-0.000369,0.077701,0.65,0.20,0.35,20,55193.3900,"Ticker(contract=Option(conId=340252339, symbol..."
450,BANKNIFTY,24700.0,1,P,27392.50,NaN,0.812706,8.90,0.000000,NaN,NaN,0.544494,-0.000411,0.084197,0.60,0.20,0.30,20,55193.3900,"Ticker(contract=Option(conId=340252345, symbol..."
453,BANKNIFTY,24400.0,1,P,27392.50,NaN,0.808140,8.85,0.000000,NaN,NaN,0.596615,-0.000287,0.063155,0.55,0.25,0.30,20,55193.3900,"Ticker(contract=Option(conId=340252322, symbol..."


In [ ]:
# filter out puts which are not in blacklist
dfs_p = dfs[(dfs.right=='P') & (~dfs.ibSymbol.isin(blacklist))]

# filter out puts whose underlying price is greater than minimum rom
df2 = dfs_p[(dfs_p.rom > min_rom)]

# filter puts whose strike price is less than the underlying's mean.
df3 = df2[(df2.strike < df2.undMean)]

In [ ]:
# WARNING!!!... Make df the dataframe that you want to execute on!
df = df3.copy()   # make this the last dataframe to get the orders placed

contracts = [c.contract for c in df.ticker]

df.loc[df.expPrice < 0.2, 'expPrice'] = 0.2  # Make the selling price a minimum of 0.2
orders = [LimitOrder(action='SELL', totalQuantity=lot, lmtPrice=expPrice) for lot, expPrice in zip(df.lot, df.expPrice)]

print('{:d} contracts consuming {:,} margin from full available funds of {:,}'.format(len(contracts), sum(df.margin), av_funds))

In [ ]:
df_analyze = df[df.ibSymbol.isin(df2.ibSymbol.unique())].groupby(['expiry', 'ibSymbol', 'strike']).apply(max)
df_analyze

In [ ]:
df_analyze.to_excel('./zdata/analyze.xlsx', index=None, header=True)

# Write to watchlist. This watchlist is to be checked in tws / tradingview for the lowest strike and expiry
# This needs to be imported to IBKR's watchlist
watchlist = [('DES', s, 'STK', 'NSE') for s in df2.ibSymbol.unique()]
df_watch = util.df(watchlist)

df_watch.to_csv('./zdata/watchlist.csv', index=None, header=False)

In [ ]:
df_analyze.ibSymbol.unique()

In [ ]:
# Write closed orders placed to file
x  =  r'./zdata/' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_closing_orders.xlsx"
writer = pd.ExcelWriter(x)
df_p.to_excel(writer, 'all-options', index=False, header=1)
writer.save()